# naive word2vec
Attack problem by: word2vec 內建的各種 functions  
其中表現最好的在 `sample dataset` 中正確率為 `50%`(比亂猜好三倍)

In [1]:
from gensim.models import word2vec
import numpy as np
from scipy import spatial

# Import & Init jieba
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re

Using TensorFlow backend.
Building prefix dict from /home/sunset/word_contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u849ecfdca27003d306f39ca004b82b5b.cache
Loading model cost 1.150 seconds.
Prefix dict has been built succesfully.


### Load datasets

In [2]:
sample = pd.read_csv('datas/sample_test_data.txt')
sample

,id,dialogue,options,answer
0,0,A:你這麼快就知道了,B:全家就是你家\tB:付出不是浪費時間\tB:願意為社會付出的人太少了\tB:我都是一個人...,4
1,1,A:每一支冰塊都不同 水質不同 硬度不同,A:紋路不同\tA:也是你唯一發洩情緒的辦法吧\tA:還記不記得那次你在我家巷口\tA:像拼...,0
2,2,A:這樣沖這麼一大塊網子 要多久時間,B:有更舒適的環境\tB:感覺是一隻很貼心的貓咪\tB:我相信一定可以成功的啦\tB:可以講...,5
3,3,A:兒子啊 都幾點了 你還不睡,B:沒關係啦 我來幫你\tB:我在背書啦 就快背好了\tB:造成誤會 不是很可惜嗎\tB:女...,1
4,4,A:孩子還在念書的時候 看到其他小朋友都有父母 而自己沒有媽媽,A:那孩子就是突然\tA:就是說\tA:禮拜一到學校\tA:媽媽還不會開車\tA:就會問我媽...,4
5,5,A:那牠吐絲這三天三夜\tA:你們不就都要一直在這邊守護著牠囉,B:涂先生\tB:那這些蠶寶寶在這邊吐絲\tB:要吐多久啊\tB:溫度三十多度的話\tB:對...,4
6,6,A:奇怪了 濂僑跑到哪兒去了呢 濂僑 你怎麼啦,B:我可能吃壞肚子了 肚子好痛喔\tB:另外一種比法是 一星期有七天\tB:為什麼不可以 同...,0
7,7,A:是什麼讓玉屏的改變這麼大啊,B:當然是紀老師的魔法囉\tB:都交給這位大力士吧\tB:粗重的交給我就對了\tB:我是八年...,0
8,8,A:曉書啊 你手上拿的 該不是最新的遊戲機吧,B:你不要小看這個扯鈴\t B:互相幫忙一下嘛\tB:一個週末出去走走 沒有關係的啦\tB:...,3
9,9,A:失戀的確會讓人家很難過,A:所以你要縮短認真難過的時間\tA:他走了十三個年頭\tA:獲選之後當然是很開心囉\tA:...,0


In [3]:
x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]
y = sample.answer.values
assert(np.sum([len(_)!=6 for _ in x2]) == 0)

In [4]:
test_datas = pd.read_csv('datas/AIFirstProblem.txt')
test_datas

,id,dialogue,options
0,1,A:媽給你送錢包來啦 來 你看一下是不是這個\tB:對 就是這個 你在哪裡找到它的,A:你看 這是我新買的錢包\tA:我的錢包不見了啦\tA:以後上網咖的錢包在我身上\tA:什...
1,2,A:古人說三日不讀書 面目可憎 我覺得我最近可能臉色太難看了,A:所以想回復我昔日面貌姣好的樣子\tA:是不是要定期來舉辦\tA:各辦理一次才對\tA:能...
2,3,A:你說我們做父母的最擔心的就是這個,A:我剛剛聽你媽說你要讀什麼科\tA:其他老師又集體叛變\tA:聽起來好好玩天啊\tA:只是...
3,4,A:是不是因為以前你的書包太重了 所以你想要發明這個,B:這個應該也是其中之一的原因\tB:好神奇喔\tB:它怎麼突然會這樣\tB:那我去探索一下...
4,5,A:那可不見得 有錢能使鬼推磨,B:好 這個就是我們今天比較級的用法\tB:真的嗎 什麼禮物\tB:有餅乾吃我也要吃\tB:...
5,6,A:真的嗎\tB:當然是真的囉 如果你不相信的話 可以問姜老師啊,C:老師常常看了都很感動\tC:你不要太在意同學們怎麼想\tC:老師先給你們一個功課\tC:...
6,7,A:你今天是要介紹哪一位人物 讓我們認識呢\tB:是和戲劇有關喔 特別是默劇,A:我們不是當事人，不太方便回答相關問題\tA:兩人得到了法國航空提供的巴黎來回機票\tA:...
7,8,A:輝哥 全部都被你吃了 你很過分\tB:就兩顆 你還剩下這樣子 那我吃什麼\tC:沒有 因...,C:然後煮一煮就變成紅色的嗎\tC:我就把它吃完了\tC:現在要把那個木頭給粉碎掉\tC:因...
8,9,A:大嬸啊 我不是說過了嗎 我們村子都是善男信女 老弱婦孺 怎麼會是練功的料呢,B:看來大家對男生女生應該有什麼樣子 還是有很刻板的印象喔\tB:我們讀不讀書關你屁事啊 你...
9,10,A:你只要拿得到滅火器 就代表你的逃生動線是安全的 你就算火滅不掉,A:防災達人接下來要突擊的\tA:是位在桃園八德市的一家雜貨店\tA:之前台語歌曲天后張秀卿...


In [5]:
test_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.dialogue.values]
test_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.options.values]
assert(np.sum([len(_)!=6 for _ in test_x2]) == 0)

### word2vec model

In [6]:
word2vec_model = word2vec.Word2Vec.load('models/word2vec_250.model.bin')
len(word2vec_model.wv.vocab)

614202

In [7]:
word2vec_model.wv.index2word[10]  # given index, find word string in word2vec_model's vocab

'小行星'

In [8]:
word2vec_model.wv.vocab['國王'].index  # given word string, find index in word2vec_model's vocab

772

In [9]:
# Count total occurance to estimate word probability
total_word_cnt = np.sum([_.count for _ in word2vec_model.wv.vocab.values()])
total_word_cnt

176304043

### Prepare for bagging

In [10]:
sample_ans = []
sample_correct = []
test_ans = []

### Naive trial - centroid

In [11]:
def centroid(sentence):
    return np.mean(
        [word2vec_model.wv.word_vec(word) for word in sentence if word in word2vec_model.wv.vocab],
        axis=0
    )

In [12]:
def attack_naive_centroid(x1, x2):
    my_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        a_center = centroid(a_sentence)
        b_centers = [centroid(s) for s in b_sentences]

        score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
        my_ans.append(np.argmin(score))
    return np.array(my_ans)

In [13]:
_ = attack_naive_centroid(x1, x2)
correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('centroid', correct, len(y)))


# Prepare for bagging
sample_ans.append(_)
sample_correct.append(correct)
test_ans.append(attack_naive_centroid(test_x1, test_x2))

            centroid:  24 /  50


### Naive trial - weighted centroid

In [14]:
def weighted_centroid(sentence, a=1e-4):
    _ = [a / (a + word2vec_model.wv.vocab[word].count / total_word_cnt) * word2vec_model.wv.word_vec(word)
            for word in sentence if word in word2vec_model.wv.vocab]
    return np.mean(_, axis=0) if len(_) > 0 else np.zeros(word2vec_model.vector_size)

In [15]:
def attack_naive_weighted_centroid(x1, x2):
    my_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        a_center = weighted_centroid(a_sentence)
        b_centers = [weighted_centroid(s) for s in b_sentences]

        score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
        my_ans.append(np.argmin(score))
    return np.array(my_ans)

In [16]:
_ = attack_naive_weighted_centroid(x1, x2)
correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('centroid', correct, len(y)))


# Prepare for bagging
sample_ans.append(_)
sample_correct.append(correct)
test_ans.append(attack_naive_weighted_centroid(test_x1, test_x2))

            centroid:  30 /  50


### Naive trial - setence embedding

In [17]:
def setence_embedding(sentence, u, a=1e-4):
    vs = weighted_centroid(sentence, a)
    return vs - u * (vs @ u)

In [18]:
u = np.load('datas/sentence_embedding/first_principle_component.npy')

def attack_naive_sentence_embedding(x1, x2):
    my_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        a_center = setence_embedding(a_sentence, u)
        b_centers = [setence_embedding(s, u) for s in b_sentences]

        score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
        my_ans.append(np.argmin(score))
    return np.array(my_ans)

In [19]:
_ = attack_naive_sentence_embedding(x1, x2)
correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('centroid', correct, len(y)))


# Prepare for bagging
sample_ans.append(_)
sample_correct.append(correct)
test_ans.append(attack_naive_sentence_embedding(test_x1, test_x2))

            centroid:  29 /  50


### Naive trial - cosine similarity between two sets of words

In [20]:
def cos_sim(a_words_set, b_words_set):
    a = [word for word in a_words_set if word in word2vec_model.wv.vocab]
    b = [word for word in b_words_set if word in word2vec_model.wv.vocab]
    if len(a) == 0 or len(b) == 0:
        return 0
    return word2vec_model.wv.n_similarity(a, b)

In [21]:
def attack_naive_cos_sim(x1, x2):
    my_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        score = [cos_sim(a_sentence, bs) for bs in b_sentences]
        my_ans.append(np.argmax(score))
    return np.array(my_ans)

In [22]:
_ = attack_naive_cos_sim(x1, x2)
correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('cos similarity', correct, len(y)))


# Prepare for bagging
sample_ans.append(_)
sample_correct.append(correct)
test_ans.append(attack_naive_cos_sim(test_x1, test_x2))

      cos similarity:  24 /  50


### Naive trial - Word Mover's Distance
- [A Linear Time Histogram Metric for Improved SIFT Matching](http://www.cs.huji.ac.il/~werman/Papers/ECCV2008.pdf)
- [Fast and Robust Earth Mover’s Distances](http://www.cs.huji.ac.il/~werman/Papers/ICCV2009.pdf)
- [From Word Embeddings To Document Distances](http://proceedings.mlr.press/v37/kusnerb15.html)

In [23]:
def attack_naive_word_mover_dis(x1, x2):
    my_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        score = [word2vec_model.wv.wmdistance(a_sentence, bs) for bs in b_sentences]
        my_ans.append(np.argmin(score))
    return np.array(my_ans)

In [24]:
_ = attack_naive_word_mover_dis(x1, x2)
correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('document dis', correct, len(y)))


# Prepare for bagging
sample_ans.append(_)
sample_correct.append(correct)
test_ans.append(attack_naive_word_mover_dis(test_x1, test_x2))

        document dis:  25 /  50


## Try bagging

In [25]:
everyone_same = (sample_ans[0] == sample_ans[1]) & (sample_ans[0] == sample_ans[2]) & (sample_ans[0] == sample_ans[3]) & (sample_ans[0] == sample_ans[4])

print('大家都對：')
print(np.arange(everyone_same.shape[0])[everyone_same & (sample_ans[0]==y)])
print('size', len(np.arange(everyone_same.shape[0])[everyone_same & (sample_ans[0]==y)]))
print('')

print('大家都錯：')
print(np.arange(everyone_same.shape[0])[
    (sample_ans[0]!=y) & (sample_ans[1]!=y) & (sample_ans[2]!=y) & (sample_ans[3]!=y) & (sample_ans[4]!=y)
])
print('size', len(np.arange(everyone_same.shape[0])[
    (sample_ans[0]!=y) & (sample_ans[1]!=y) & (sample_ans[2]!=y) & (sample_ans[3]!=y) & (sample_ans[4]!=y)
]))
print('')

大家都對：
[ 1 14 18 19 21 22 23 26 27 30 32 33 36 44]
size 14

大家都錯：
[ 0  3  7  8 20 29 34 37 39 41 48 49]
size 12



In [50]:
sample_voter = np.vstack([
    np.repeat([sample_ans[i]], sample_correct[i], axis=0)
    for i in [0, 1]
])

correct = 0
for i in range(len(y)):
    if np.argmax(np.bincount(sample_voter[:, i])) == y[i]:
        correct += 1

print('%20s: %3d / %3d' % ('bagging', correct, len(y)))

             bagging:  30 /  50


## Output answer on testing datas

In [27]:
test_voter = np.vstack([
    np.repeat([test_ans[i]], sample_correct[i], axis=0)
    for i in range(len(test_ans))
])

In [28]:
my_test_ans = [np.argmax(np.bincount(test_voter[:, i])) for i in range(len(test_x1))]

In [29]:
for i in range(len(test_x1)):
    print('questions:', test_x1[i])
    for j, option in enumerate(test_x2[i]):
        print('\t%d) %s' % (j, option))
    print('my answer:', my_test_ans[i])
    print('=' * 100)

questions: ['媽給你送錢包來啦 來 你看一下是不是這個', '對 就是這個 你在哪裡找到它的']
	0) 你看 這是我新買的錢包
	1) 我的錢包不見了啦
	2) 以後上網咖的錢包在我身上
	3) 什麼有錢包場
	4) 早上你爸爸在車上找到的 一定是前天你放學的時候掉在車上了
	5) 我為什麼要給你們錢包
my answer: 4
questions: ['古人說三日不讀書 面目可憎 我覺得我最近可能臉色太難看了']
	0) 所以想回復我昔日面貌姣好的樣子
	1) 是不是要定期來舉辦
	2) 各辦理一次才對
	3) 能夠督促所有的用人機關
	4) 在上次的節目討論中也有提到
	5) 超過九十分貝以上
my answer: 0
questions: ['你說我們做父母的最擔心的就是這個']
	0) 我剛剛聽你媽說你要讀什麼科
	1) 其他老師又集體叛變
	2) 聽起來好好玩天啊
	3) 只是小孩自己的興趣不能得到發展 他們的心裡可能也會很悶喔
	4) 走到這裡就沒有路了耶
	5) 每一個科目像是國語數學都很優秀
my answer: 3
questions: ['是不是因為以前你的書包太重了 所以你想要發明這個']
	0) 這個應該也是其中之一的原因
	1) 好神奇喔
	2) 它怎麼突然會這樣
	3) 那我去探索一下
	4) 牠會不會覺得很不開心 會生氣
	5) 我覺得很高興
my answer: 0
questions: ['那可不見得 有錢能使鬼推磨']
	0) 好 這個就是我們今天比較級的用法
	1) 真的嗎 什麼禮物
	2) 有餅乾吃我也要吃
	3) 我知道隔壁班今天也有地理課 我陪你去借
	4) 什麼你說我是鬼
	5) 我表姊從國外帶回來的 送你一個
my answer: 0
questions: ['真的嗎', '當然是真的囉 如果你不相信的話 可以問姜老師啊']
	0) 老師常常看了都很感動
	1) 你不要太在意同學們怎麼想
	2) 老師先給你們一個功課
	3) 就是因為這樣 你們才要更包容他更鼓勵他
	4) 是啊 這點老師也可以證明喔
	5) 沒有人是十全十美的
my answer: 4
questions: ['你今天是要介紹哪一位人物 讓我們認識呢', '是和戲劇有關喔 特別是默劇']
	0) 我們不是當事人，不太方便回答相關問題
	1

	2) 這個一個五塊
	3) 我的兒女是很捨不得啊
	4) 不過我還是覺得很興奮
	5) 看到很多人的單車日記
my answer: 3
questions: ['你別撒嬌了 你早知道我會幫你抄聯絡簿對不對']
	0) 被你發現囉
	1) 我也好想念一起回家的日子
	2) 九月十日
	3) 那個時候 我們一起走在同一條路上
	4) 我很喜歡這個闕詞
	5) 蕙馨得到第一名
my answer: 1
questions: ['我真的沒有想過要傷害你', '我同學說的那些話是無心的']
	0) 半夜三更不睡覺
	1) 有什麼好洗的啊
	2) 你會一點點手語耶
	3) 你交了別的女朋友
	4) 請你不要放在心上
	5) 他越是有可能回來找你的啦
my answer: 5
questions: ['為什麼要領錢還要事先講呢', '那是因為行動郵局 它不會帶太多錢出門喔', '要怎麼領呢']
	0) 他可以重複領好幾個地方
	1) 老闆娘 你要領那麼多
	2) 看你笑得眉開眼笑的
	3) 我回去打打看因為你超過時間了
	4) 由行動郵局的人開車載你回郵局領錢
	5) 所以就算是要存錢也不行
my answer: 5
questions: ['大家叫他銼冰泉 因為他在附近一家製冰工廠切冰塊 要是那一天冰塊切得很順']
	0) 我等這一刻等很久了
	1) 你給我飲料幹什麼啦
	2) 我看到金剛的眼睛
	3) 贏了比賽就不能快樂的跳舞
	4) 決戰到最後的時刻
	5) 他就會在外頭喝兩杯再回家
my answer: 5
questions: ['又熬夜K書了 讀不完了 對不對', '不要吵 我快背好了', '明天上學不要爬不起來 遲到']
	0) 你願意原諒我們嗎
	1) 你們在看什麼
	2) 沒去嘗試怎麼知道不會呢
	3) 你就是接受我的邀請囉
	4) 我有調鬧鐘啦
	5) 我不應該發那麼大的脾氣的
my answer: 1
questions: ['可能是最近交了壞朋友 所以才變壞的', '對啊 碧珊你趕快跟劉老師說對不起 請他原諒你', '你們幹嘛這樣低聲下氣啊 他是罵我沒家教 又不是罵你們']
	0) 你罵夠了沒啊 考零分就考零分 你憑什麼罵我沒家教
	1) 碰到不會寫的題目 至少要填個答案上去
	2) 簡直就是小太妹嘛 我們以後要跟她保持距離才好
	3) 你嘴巴厲害 我倒

In [30]:
with open('answer/attack-naive-word2vec-bagging.txt', 'w') as fo:
    fo.write('id,ans\n')
    fo.write('\n'.join(['%d,%s' % (i+1, ans) for i, ans in enumerate(my_test_ans)]))
    fo.write('\n')

In [31]:
# Compare with previos result
backup_ans = pd.read_csv('answer/attack-naive-word2vec.txt')
backup_ans = backup_ans.ans.values

for i in range(len(test_x1)):
    if my_test_ans[i] == backup_ans[i]:
        continue
    print('questions:', test_x1[i])
    for j, option in enumerate(test_x2[i]):
        print('\t%d) %s' % (j, option))
    print('now    answer:', my_test_ans[i])
    print('backup answer:', backup_ans[i])
    print('=' * 100)

questions: ['媽給你送錢包來啦 來 你看一下是不是這個', '對 就是這個 你在哪裡找到它的']
	0) 你看 這是我新買的錢包
	1) 我的錢包不見了啦
	2) 以後上網咖的錢包在我身上
	3) 什麼有錢包場
	4) 早上你爸爸在車上找到的 一定是前天你放學的時候掉在車上了
	5) 我為什麼要給你們錢包
now    answer: 4
backup answer: 0
questions: ['是不是因為以前你的書包太重了 所以你想要發明這個']
	0) 這個應該也是其中之一的原因
	1) 好神奇喔
	2) 它怎麼突然會這樣
	3) 那我去探索一下
	4) 牠會不會覺得很不開心 會生氣
	5) 我覺得很高興
now    answer: 0
backup answer: 4
questions: ['大嬸啊 我不是說過了嗎 我們村子都是善男信女 老弱婦孺 怎麼會是練功的料呢']
	0) 看來大家對男生女生應該有什麼樣子 還是有很刻板的印象喔
	1) 我們讀不讀書關你屁事啊 你管那麼多幹嘛
	2) 錯了 我們的武功是要教大家 以小博大 以柔克剛 男女老幼都可以學習的
	3) 所以做母親的 我們覺得非常了不起非常偉大
	4) 這次你就假裝沒看到
	5) 好 就交給你們
now    answer: 2
backup answer: 4
questions: ['你不知道那些狗仔 什麼時候又跑去 挖你那間舊房子的新聞']
	0) 這樣的工作不一定就是相對弱勢喔
	1) 不了解他人的利他 反而會造成更大的傷害
	2) 指考成績出爐
	3) 好啦 沒關係 心情放輕鬆一點
	4) 如果有工作後的學長姐分享會更棒
	5) 謝謝一如既往收看影片
now    answer: 0
backup answer: 3
questions: ['其實你可以試著 把你最近感覺到的 還有想到的 把它寫下來']
	0) 可是在詩人的眼裡 下雨也有另外一番風情
	1) 真倒楣 差一點點就趕上了
	2) 你家不是很近嗎 有沒有興趣用走的
	3) 好吧 我試試
	4) 我實在是不知道女生這樣 會哭得這麼厲害
	5) 你還要當著全班的面朗讀 她當然會覺得很難堪
now    answer: 5
backup answer: 2
questions: ['說台灣每年大概有四千人 因為自殺而